In [32]:
from astrosite_dataset import AstrositeDataset

dataset_path = 'data/astrosite/recordings'
dataset = AstrositeDataset(dataset_path, split='37867')

In [4]:
from itertools import islice

n_samples = 9
samples = [sample['labelled_events'] for sample in islice(dataset, n_samples)]

In [33]:
samples = []
n_samples = 9

for sample in dataset:
    labels = sorted(list(set(sample['labelled_events']['label'])))
    if labels[0] < -1:
        print(f"Skipping sample with labels {labels}")
        continue
    
    events = sample['labelled_events']
    sat_events = events[events['label'] == -1]
    samples.append(sat_events)
    if len(samples) >= n_samples:
        break

Skipping sample with labels [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]


In [34]:
import numpy as np

side_length = np.sqrt(n_samples).astype(int)
max_x = 1280
max_y = 720

for i in range(side_length):
    for j in range(side_length):
        index = i*side_length+j
        x_shift = max_x*j
        y_shift = max_y*i
        samples[index]['x'] += x_shift
        samples[index]['y'] += y_shift

merged_events = np.concatenate(samples)
merged_events = np.sort(merged_events, order='t')

In [35]:
import event_stream

encoder = event_stream.Encoder('merged_events.es', 'dvs', 1280*side_length, 720*side_length)

In [36]:
encoder.write(merged_events)